In [7]:
import numpy as np
import os
import tarfile
import pickle
import platform
import numpy as np
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


# 加载训练数据
def load_cifar10_datasets(data_dir):
    # CIFAR-10 数据集包含 5 个训练批次和 1 个测试批次
    train_data = []
    train_labels = []

    # 加载每个训练批次
    for i in range(1, 6):  # data_batch_1 到 data_batch_5
        #batch = unpickle(f'{data_dir}/data_batch_{i}')
        with open(f'{data_dir}/data_batch_{i}', 'rb') as fo: 
            batch = pickle.load(fo, encoding='bytes') 
        train_data.append(batch[b'data'])
        train_labels.append(batch[b'labels'])

    # 合并数据
    x_train = np.vstack(train_data)  # 将所有批次的数据合并为一个大数组
    y_train = np.concatenate(train_labels)  # 合并所有批次的标签

    # 加载测试数据
    test_batch = unpickle(f'{data_dir}/test_batch')
    x_test = test_batch[b'data']
    y_test = np.array(test_batch[b'labels'])

    # 返回训练和测试数据及其标签
    return (x_train, y_train), (x_test, y_test)


def to_categorical(y, num_classes):
    """从keras中复制而来
    Converts a class vector (integers) to binary class matrix.

    E.g. for use with categorical_crossentropy.

    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.

    # Returns
        A binary matrix representation of the input. The classes axis
        is placed last.
    """
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=np.float32)
    categorical[np.arange(n), y] = 1
    return categorical
train_set,  test_set = load_cifar10_datasets('cifar-10-python/cifar-10-batches-py/')
train_y,test_y=to_categorical(train_set[1],10),to_categorical(test_set[1],10)


# 损失函数：均方误差
def mean_squared_loss(y_pred, y_true):
    """
    计算均方误差损失及其梯度
    :param y_pred: 预测值，形状为 (N, d)
    :param y_true: 真实值
    :return: 损失和损失函数的梯度
    """
    loss = np.sum(np.square(y_pred - y_true), axis=-1)  # 计算损失值
    dy = 2 * (y_pred - y_true) / y_pred.shape[0]  # 损失函数关于输出的梯度
    return loss, dy

# 全连接层前向传播
def fc_forword(x, W, b):
    """
    计算全连接层的前向传播
    :param x: 输入，形状为 (N, d)
    :param W: 权重，形状为 (d, k)
    :param b: 偏置，形状为 (1, k)
    :return: 全连接层的输出
    """
    return np.dot(x, W) + b

def fc_backword(dy, x, z):
    """
    全连接层的反向传播计算
    :param dy: 损失函数对输出的导数，形状为 (N, 10)
    :param x: 输入，形状为 (N, 3072)
    :param z: 前向传播时的线性组合 z，形状为 (N, 10)
    :return: 对偏置和权重的梯度
    """
    dz = np.zeros_like(z)
    dz[z > 0] = 1  # ReLU 的导数： z > 0 的导数为 1

    # 计算对偏置和权重的梯度，确保维度正确
    db = np.sum(dy * dz, axis=0, keepdims=True) / x.shape[0]  # (1, 10)
    dw = np.dot(x.T, dy * dz) / x.shape[0]  # (3072, 10)

    return db, dw


import numpy as np
# 初始化权重 W，形状为 (3072, 10)，使用随机值
W = np.random.randn(3072, 10)*0.01
# 初始化偏置 b，形状为 (1, 10)，全部赋值为0
b = np.zeros((1, 10))
# 随机选择训练样本
train_num = train_set[0].shape[0]
def next_batch(batch_size):
    idx=np.random.choice(train_num,batch_size)
    return train_set[0][idx],train_y[idx]
# 随机初始化参数
loss = 100.0
lr = 0.0000000000000001
i = 0 
for i in range(1000):
    x,y= next_batch(32)
    # 前向传播
    z = fc_forword(x,W,b)
    y = np.maximum(0, z)
    # 反向传播更新梯度
    loss,dy=mean_squared_loss(y,train_y[i])
    db,dw = fc_backword(dy,x,z)
    
    # 在一个batch中梯度取均值
    #print(dw)
    
    # 更新梯度
    W += lr*dw
    b += lr*db
    # 在前 10 步输出 dw 和 db
    if i < 10:
        print(f"Step {i}:")
        print(f"  Gradient of W (dw): \n{dw}")
        print(f"  Gradient of b (db): \n{db}")
        print("-" * 50)
        print(f"  Current Loss: {loss}")


correct_predictions = 0
x_test = test_set[0] 
# 遍历测试集
for i in range(x_test.shape[0]):
    x = x_test[i]
    x = x.reshape(3072, 1)  # 变为 (3072, 1)
    x = x.T                 # 转置为 (1, 3072)
    # 前向传播
    z = fc_forword(x, W, b)  # 假设 fc_forword 函数已经定义
    y = np.maximum(0, z)      # ReLU 激活

    # 预测类别
    predicted_class = np.argmax(y)  # 最大值的索引

    # 实际类别（one-hot）
    actual_class = np.argmax(test_y[i])  # 获取实际标签的类别

    # 计数正确预测
    if predicted_class == actual_class:
        correct_predictions += 1
    if i<100:
        print(predicted_class,actual_class)
    
# 计算准确率
accuracy = correct_predictions / x_test.shape[0]
print(f"Accuracy: {accuracy * 100:.2f}%")

Step 0:
  Gradient of W (dw): 
[[ 15.34700244 187.13998886   0.75275981 ...  88.30273489 213.65847992
  348.78039701]
 [  6.41783738 187.58761504   0.7945798  ...  93.1194749  212.39211352
  360.61148343]
 [ 13.04495207 187.81270442   0.85730978 ...  90.69887847 210.90924754
  354.90651308]
 ...
 [ 13.53326579  87.25230438   2.07008947 ...  59.10147327 171.2353444
  295.47816719]
 [ 12.41712016  90.32203696   2.04917947 ...  62.28006773 175.81513902
  300.76705736]
 [ 11.92880644  91.45564962   2.04917947 ...  63.22246293 173.75036163
  297.83531177]]
  Gradient of b (db): 
[[0.0697591  1.07401608 0.02090999 1.14389231 1.09343923 0.
  7.69847777 0.62255094 1.11434949 2.62170888]]
--------------------------------------------------
  Current Loss: [13215.83142497 22256.2503682  49128.65668064 65051.92208309
 13369.66622467  9255.23015412 12620.79218606 45341.72611594
 13948.26704958 16401.80037498 21945.607405   13374.73271775
 26500.0269775  27713.80692299 30126.62288123 32578.5126923
 